In [1]:
import pandas as pd

# Composition


In [2]:
def compose(*functions):
    # return a function that takes data
    def inner(data):
        for f in functions:
            data = f(data)
        return data

    return inner

In [3]:
def multiply_by_two(x):
    return x * 2


def divide_by_two(x):
    return x / 2


def add_one(x):
    return x + 1

In [4]:
x = 0

divide_by_two(add_one(multiply_by_two(multiply_by_two(add_one(x)))))

2.5

In [5]:
compose(add_one, multiply_by_two, multiply_by_two, add_one, divide_by_two)(x)

2.5

# Refactor the code by using composition

Code with Pyramid of Doom

```python
def process_bmi(data_file: str, weight_kg_col: str, height_cm_col: str) -> pd.DataFrame | None:
    # load data
    # read weight column
    # read height column
    # calculate BMI
    # return df
    try:
        df = pd.read_csv(data_file)
        try:
            weights = df[weight_kg_col]
            try:
                heights = df[height_cm_col]
                try:
                    bmi = weights / (heights / 100) ** 2
                    df['bmi'] = bmi.round(1)
                    return df
                except Exception as e:
                    print(f"Error calculating BMI: {e}")
                    return None
            except Exception as e:
                print(f"Error reading height column: {e}")
                return None
        except Exception as e:
            print(f"Error reading weight column: {e}")
            return None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
```


## Functions should do only one thing and do it well


In [6]:
def read_df(path: str) -> pd.DataFrame:
    return pd.read_csv(path)


def convert_height_cm_to_m(height_cm_col: str, target_col: str):
    def convert(df: pd.DataFrame) -> pd.DataFrame:
        df[target_col] = df[height_cm_col] / 100
        return df

    return convert


def calculate_bmi(weight_kg_col: str, height_m_col: str, target_col: str):
    def calculate(df: pd.DataFrame) -> pd.DataFrame:
        df[target_col] = round(df[weight_kg_col] / (df[height_m_col] ** 2), 2)
        return df

    return calculate

### better?


In [7]:
csv_path = "data/good.csv"

df = read_df(csv_path)
df = convert_height_cm_to_m("height_cm", "height_m")(df)
df = calculate_bmi("weight", "height_m", "bmi")(df)
df

,name,height_cm,weight,height_m,bmi
0,Alice,170,55,1.70,19.03
1,Bob,180,70,1.80,21.60
2,Charlie,175,65,1.75,21.22
3,David,203,110,2.03,26.69
4,Eve,173,60,1.73,20.05
5,Frank,155,110,1.55,45.79
6,Grace,169,45,1.69,15.76
7,Hannah,165,50,1.65,18.37
8,Ivan,175,70,1.75,22.86
9,John,210,95,2.10,21.54


### using composition pattern


In [8]:
df_bmi = compose(
    read_df,
    convert_height_cm_to_m("height_cm", "height_m"),
    calculate_bmi("weight", "height_m", "bmi"),
)(csv_path)
df_bmi

,name,height_cm,weight,height_m,bmi
0,Alice,170,55,1.70,19.03
1,Bob,180,70,1.80,21.60
2,Charlie,175,65,1.75,21.22
3,David,203,110,2.03,26.69
4,Eve,173,60,1.73,20.05
5,Frank,155,110,1.55,45.79
6,Grace,169,45,1.69,15.76
7,Hannah,165,50,1.65,18.37
8,Ivan,175,70,1.75,22.86
9,John,210,95,2.10,21.54
